In [19]:
import pandas as pd
import requests
import json
import openai

from config import secret_key_api_chat_gpt

EXTRACT

In [20]:
df = pd.read_csv('SDVW2023.csv')

user_id = df['UserID'].to_list()

print(user_id)

[1, 2, 3, 4, 5]


In [21]:
sdw_api = 'https://sdw-2023-prd.up.railway.app'

def get_user(id):
    response = requests.get(f'{sdw_api}/users/{id}') 
    return response.json() if response.status_code == 200 else None



users = [user for id in user_id if (user := get_user(id)) is not None]



TRANSFORM

In [ ]:
#https://platform.openai.com/account/api-keys
#https://platform.openai.com/docs/api-reference/introduction?lang=python
openai.api_key = secret_key_api_chat_gpt


def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Você é um coach."},
        {"role": "user", "content": f"Crie uma mensagem para {user['name']} de ânimo (com máximo 100 caracteres)"}
    ]
    )

    return completion.choices[0].message.content.strip('\"')


for user in users:
    new = generate_ai_news(user)
    user['news'].append({
        'description': new
    })



Load

In [ ]:
def update_user(user):
    response = requests.put(f'{sdw_api}/users/{user["id"]}', json=user)
    return True if response.status_code == 200 else False



for user in users:
    sucess = update_user(user)
    print(f'User {user["name"]} updated? {sucess}')